# Jupyter and IPython display protocol

In [ ]:
import pandas as pd
import seaborn.apionly as sns
iris = sns.load_dataset('iris')

In [ ]:
iris.head()

## Modifying objects reprs

In [ ]:
class MultiMime:
    
    def __repr__(self):
        return "this is the repr"
    
    def _repr_html_(self):
        return "This <b>is</b> html"
    
    def _repr_markdown_(self):
        return "This **is** markdown"

    def _repr_latex_(self):
        return "$ Latex \otimes mimetype $"

In [ ]:
display(MultiMime())

In [ ]:
MultiMime()

In [ ]:
# restart the kernel

MultiMime() # SHould fail with kernel restarted

## External formatters

As a visual example we'll use Orly Parody books covers, in particular a small resolution of some of them to limit the amount of data we'll be working with. 

In [ ]:
cd thumb

Let's see some of the images present in this folder:

In [ ]:
names = !ls *.png
names[:20], f"{len(names) - 10} more"

In [ ]:
from IPython.display import Image

In [ ]:
im = Image(names[0])
im

In [ ]:
from random import choices
mylist = list(map(Image, set(choices(names, k=10))))
mylist

In [ ]:
import base64
def tag_from_data(data, size='100%'):
    return '''<img
                    style="
                        display:inline;
                        width:{1};
                        max-width:400px;
                        padding:10px;
                        margin-top:14px"
                    onMouseOver="this.style['box-shadow']='5px 5px 30px 0px rgba(163,163,163,1)'" 
                    onMouseOut="this.style['box-shadow']=''"
                    src="data:image/png;base64,{0}" 
             />'''.format(''.join(base64.encodebytes(data).decode().split('\n')), size)

def html_list_formatter(ll):
    html = get_ipython().display_formatter.formatters['text/html']
    reps = []
    for o in ll:
        if isinstance(o, Image):
            reps.append(tag_from_data(o.data, '200px') )
        else: 
            h = html(o)
            if h:    
                reps.append(h)
            else:
                reps.append(repr(o)+'')
    
    return '<span>['+','.join(reps)+']</span>'

Same as before, with square bracket after and before, and a bit of styling that change the drop shadow on hover. Now we register the above with IPython:

In [ ]:
mylist

In [ ]:
ipython = get_ipython()
html_formatter = ipython.display_formatter.formatters['text/html']
html_formatter.for_type(list, html_list_formatter)

In [ ]:
mylist

## Disp

External integration for some already existing object is available in [disp](https://github.com/ipython/disp), in particular you will find representation for SparkContext, `requests`'s `Responses` object (collapsible json content and headers), as well as a couple others. 

In [ ]:
from requests.models import Response
import requests

In [ ]:
r = requests.get('https://api.github.com') # http://localhost:8888/api/contents is down/no-network
r

In [ ]:
r.json()

### Existing formatters

In [ ]:
from disp.py3only import html_formatter_for_Response
html_formatter.for_type(Response, html_formatter_for_Response)

In [ ]:
r